In [1]:
import plotly.express as px
import sklearn as sk
import pandas as pd
import numpy as np
import random


In [2]:
actions_data = pd.read_csv("../data/actions.csv").dropna()
cars_data = pd.read_csv("../data/cars_about.csv").dropna()

In [3]:
cars_data

,car_id,car_model,used_label,price,exteriorColor,interiorColor,drivetrain,minMPG,maxMPG,fuelType,transmission,engine,mileage
0,1.0,Toyota Sienna SE,Used,$39,Red,Black,Front-wheel Drive,19.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,29403.0
1,2.0,Ford F-150 Lariat,Used,$49,Shadow Black,Black,Four-wheel Drive,19.0,24.0,Gasoline,10-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,32929.0
2,3.0,RAM 1500 Laramie,Used,$41,Granite Crystal Clearcoat Metallic,Black,Four-wheel Drive,15.0,21.0,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,23173.0
3,4.0,Honda Accord Sport SE,Used,$28,Gray,–,Front-wheel Drive,29.0,35.0,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,10598.0
4,5.0,Lexus RX 350,Used,$49,Eminent White Pearl,Birch,Front-wheel Drive,20.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,28137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,401.0,Jeep Grand Cherokee Limited,Used,$33,Diamond Black,Black,Four-wheel Drive,18.0,25.0,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,43807.0
400,402.0,Toyota RAV4 Hybrid XLE,Toyota Certified,$30,Super White,Nutmeg,All-wheel Drive,34.0,30.0,Hybrid,Automatic CVT,2.5L I4 16V MPFI DOHC Hybrid,41944.0
401,403.0,Cadillac XT4 Sport,Used,$35,Black,Black,All-wheel Drive,22.0,29.0,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,49386.0
402,404.0,Chevrolet Tahoe LTZ,Used,$35,Tungsten Metallic,Jet Black,Four-wheel Drive,16.0,22.0,Gasoline,6-Speed Automatic,5.3L V8 16V GDI OHV,115795.0


In [4]:
actions_data

,car_id,user_id
0,84,0
1,171,0
2,80,0
3,126,0
4,184,0
...,...,...
2869,13,74
2870,17,74
2871,17,74
2872,16,74


In [5]:
actions_data = actions_data.astype(int)

In [6]:
user_data = pd.DataFrame(
    columns = sorted(actions_data.car_id.unique(), key=int), 
    index=actions_data.user_id.unique()
    )
user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


У меня есть действия пользователей в которые входит только информация о том лайкнул он авто или нет, мне кажется, что это довольно скупая информация. Для того чтобы диверсифицировать предпочтения сгенерированных пользователей я буду рандомно генерировать рейтинги для фильмов <br> Каким образом? <br> авто было добавлено в любимые? => random([3, 4, 5]) <br> С остальными авто поступаю иначе, генерирую для пользователя рандомное кол-во авто с низким рейтингом и уже эти авто раздаю рейтинги от 1-2

In [7]:
for user_id in actions_data.user_id:
    current_user_actions = actions_data.query(f"user_id == {user_id}")
    for car_id in current_user_actions.car_id:
        user_data[int(car_id)].iloc[user_id] = random.randint(4, 5) 

user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,NaN,NaN,NaN,5,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,NaN,4,4,5,4,4,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5,NaN,4,4,5,4,4,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
num_unique_cars = len(user_data.columns)

for row_idx in user_data.index:
    # specify the number of NaN values to replace with random numbers
    # from 0 to number of non-defined cars
    n = random.randint(0, num_unique_cars - user_data.iloc[row_idx].notna().sum())
    # get the columns with NaN values in the specified row
    cols_with_nans =  user_data.iloc[row_idx][ user_data.iloc[row_idx].isnull()].index.tolist()

    # randomly fill 'n' of the NaN values in the specified row with values between 0 and 10
    random_values = np.random.randint(1, 3, n)
    for col in np.random.choice(cols_with_nans, n, replace=False):
        user_data.loc[row_idx, col] = random_values[n - 1]

In [9]:
user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,5
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,4,2,2,2,5,2,2
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,1,4,4,5,4,4,4,1,...,NaN,1,1,1,1,NaN,1,1,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5,NaN,4,4,5,4,4,5,4,NaN,...,1,1,1,NaN,1,NaN,1,1,NaN,1
71,2,NaN,NaN,NaN,NaN,2,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,2,2,2,NaN,2,2
72,NaN,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN
73,1,1,1,1,1,1,1,1,1,1,...,1,5,1,4,1,1,1,1,1,1


In [10]:
user_data.to_csv("../data/user_data.csv", index=False)

In [11]:
matrix = user_data.div(user_data.sum(axis=1), axis=0)
# matrix = user_data

In [12]:
matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,0.019231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.019231,0.019231,NaN,0.019231,NaN,NaN,NaN,NaN,NaN,0.096154
1,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,...,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333
2,0.012739,0.012739,0.012739,0.012739,0.012739,0.012739,0.012739,0.012739,0.012739,0.012739,...,0.012739,0.012739,0.012739,0.025478,0.012739,0.012739,0.012739,0.031847,0.012739,0.012739
3,NaN,NaN,0.03125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.125,NaN,0.15625,NaN,NaN,NaN,NaN,NaN,NaN
4,0.033333,0.033333,0.008333,0.033333,0.033333,0.041667,0.033333,0.033333,0.033333,0.008333,...,NaN,0.008333,0.008333,0.008333,0.008333,NaN,0.008333,0.008333,0.041667,NaN


In [13]:
users_pierson_correlation = matrix.fillna(0).T.corr()

In [14]:
fig = px.imshow(users_pierson_correlation)
fig.show()

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

user_cosine_similarity = cosine_similarity(matrix.fillna(0))
px.imshow(user_cosine_similarity)

In [16]:
picked_user_id = 1
users_pierson_correlation.drop(index= picked_user_id, inplace=True)
users_pierson_correlation.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,1.000000,0.221017,-0.014678,0.113418,-0.229480,-0.010875,0.095385,-0.148153,0.087480,0.351761,...,-0.130561,0.154692,0.377233,0.153220,0.224893,-0.160530,0.115371,0.394284,-0.039011,-0.174341
2,-0.014678,-0.121885,1.000000,0.238888,-0.017164,0.244639,0.280959,0.302791,0.089168,0.036588,...,0.178993,0.224856,-0.120476,0.296941,0.498238,-0.058007,-0.086892,0.077908,0.068382,-0.045462
3,0.113418,-0.103316,0.238888,1.000000,-0.164597,0.038773,0.182644,0.229549,0.298288,0.394171,...,0.052478,0.298872,0.184753,0.042241,0.236794,-0.104555,0.075487,-0.071526,0.551585,-0.095262
4,-0.229480,-0.198408,-0.017164,-0.164597,1.000000,-0.100626,-0.187435,-0.130538,-0.175158,-0.222234,...,0.036378,-0.194055,-0.145697,0.031936,-0.122031,0.560957,-0.061783,-0.137029,-0.098784,0.684581
5,-0.010875,-0.043101,0.244639,0.038773,-0.100626,1.000000,0.107089,0.241862,-0.101975,0.088290,...,0.047325,0.019180,-0.118621,0.049449,0.118082,-0.189459,0.137535,0.264285,0.123875,-0.059121


In [17]:
top_n_users = 10
user_similarity_threshold = 0.3
similar_users = users_pierson_correlation[users_pierson_correlation[picked_user_id] > user_similarity_threshold]

In [18]:
similar_users

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
8,0.087480,0.347604,0.089168,0.298288,-0.175158,-0.101975,0.337438,-0.088222,1.000000,-0.026576,...,0.257264,0.296562,0.458903,0.043094,0.082278,-0.149308,-0.010933,0.208313,0.412313,-0.166188
29,0.112787,0.407420,-0.104258,0.085381,-0.143926,0.104265,-0.077361,0.217775,-0.063389,0.189812,...,-0.114431,0.067711,-0.074417,0.216442,-0.195486,-0.101557,0.070399,-0.033976,-0.093656,-0.088890
31,0.364879,0.513054,0.284216,0.227574,-0.207825,0.017478,0.197527,0.072241,0.236960,0.297289,...,0.022109,0.404929,0.355249,0.120421,0.281240,-0.180324,-0.040523,0.167135,0.311143,-0.048130
53,0.213619,0.317272,0.115515,-0.029098,-0.144891,0.083763,0.048148,-0.041421,0.101699,0.051936,...,0.151483,0.296513,0.231895,0.150256,0.136520,-0.190970,-0.286982,0.136397,-0.073300,-0.003510
54,0.187729,0.455228,0.152583,0.055145,-0.206568,-0.157903,0.202630,-0.065796,0.288177,0.095929,...,-0.230187,0.267176,0.029280,0.198343,-0.041440,-0.093795,-0.017552,-0.115619,-0.000342,-0.052048


In [19]:
# Movies that the target user has watched
picked_userid_watched = matrix[matrix.index == picked_user_id].dropna(axis=1, how='all')
picked_userid_watched

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
1,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,...,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333


In [20]:
# Movies that similar users watched. Remove movies that none of the similar users have watched
similar_user_movies = matrix[matrix.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_movies

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
8,0.012048,0.012048,0.012048,0.012048,NaN,0.012048,0.012048,0.012048,0.012048,0.012048,...,0.012048,0.048193,0.012048,0.012048,0.012048,0.012048,0.012048,0.048193,0.012048,0.012048
29,NaN,NaN,0.026316,0.026316,NaN,NaN,0.026316,0.026316,NaN,NaN,...,0.026316,0.026316,0.026316,NaN,0.026316,0.026316,0.026316,NaN,0.026316,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.147059,NaN,0.147059,NaN,NaN,NaN,NaN,NaN,NaN
53,NaN,0.016667,0.016667,0.016667,0.016667,NaN,0.016667,0.016667,0.016667,0.016667,...,0.016667,0.016667,0.016667,0.016667,NaN,0.016667,0.016667,0.016667,0.016667,0.016667
54,0.032787,NaN,0.032787,0.032787,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.032787,0.032787,NaN,NaN,0.081967,0.032787,0.081967


In [21]:
# Remove the watched movie from the movie list
similar_user_movies.drop(picked_userid_watched.columns,axis=1, inplace=True, errors='ignore')
# Take a look at the data
similar_user_movies

,13,45,59
8,0.012048,0.012048,0.012048
29,NaN,NaN,NaN
31,NaN,NaN,NaN
53,NaN,0.016667,0.016667
54,NaN,NaN,0.032787


In [22]:
# A dictionary to store item scores
item_score = {}
# Loop through items
for i in similar_user_movies.columns:
  # Get the ratings for movie i
  movie_rating = similar_user_movies[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in similar_users.index:
    # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
      # Score is the sum of user similarity score multiply by the movie rating
      score = sum(similar_users[u] * movie_rating[u])
      # Add the score to the total score for the movie so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the item
  item_score[i] = total / count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
# Select top m movies
m = 10
ranked_item_score.head(m)

,movie,movie_score
2,59,0.036800
1,45,0.023618
0,13,0.018837


In [23]:
cars_data.loc[cars_data.car_id.isin(ranked_item_score.movie) == True]

,car_id,car_model,used_label,price,exteriorColor,interiorColor,drivetrain,minMPG,maxMPG,fuelType,transmission,engine,mileage
12,13.0,Lexus RX 350 RX 350,Used,$36,Nightfall Mica,Gray,Front-wheel Drive,20.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V GDI DOHC,47729.0
44,45.0,Honda Civic EX,Used,$22,Modern Steel Metallic,Black / Gray,Front-wheel Drive,31.0,40.0,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,35257.0
